In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from math import factorial

In [2]:
# read data
sigmadf = pd.read_excel("Extra_Credit_Assignment_Data.xlsx", header = None, sheet_name= 0)
dtdf = pd.read_excel("Extra_Credit_Assignment_Data.xlsx", header = None, sheet_name= 1)

In [3]:
def r_to_dt(r):
    return (1 / (1 + r))

def nCr(n, r):
    return (factorial(n) / factorial(r) / factorial(n - r))

def pascalTriWeightedAverage(arr):
    n = len(arr)
    if n == 0:
        return 0
    if n == 1:
        return arr[0]

    res = 0.0
    for i in range(n):
        res += nCr(n - 1, i) / 2**(n - 1) * arr[i]
    return res

In [4]:
def func(guess, targetNode):
    """
    Iterative function to pass to solver to solve for r at each node
    """
    curr = targetNode
    n = curr.level
    r_curr = guess
    dt_target = curr.dt
    deltat = np.sqrt(0.5)
    somearr = []
    for i in range(n):
        somearr.append(0.5 / (1 + r_curr / 2) + 0.5 / (1 + r_curr * np.exp(2 * curr.sigma * deltat) / 2))
        r_curr = r_curr * np.exp(2 * curr.sigma * deltat)

    curr = curr.prev
    # while loop til root
    while curr.prev:
        r_curr = curr.r
        n = curr.level
        for i in range(n):
            # the somearr is like a reverse pyramid and only first n->...->6->5->...->2 elements will be used in the while loop
            somearr[i] = (0.5 / (1 + r_curr / 2) * somearr[i] + 0.5 / (1 + r_curr * np.exp(2 * curr.sigma * deltat) / 2) * somearr[i + 1])
            r_curr = r_curr * np.exp(2 * curr.sigma * deltat)
        curr = curr.prev

    return somearr[0] * curr.dt - dt_target

In [5]:
# tree with doubly linked list implementation since its recombining tree and each r can be computed using the same level r and sigma
class node:
    def __init__(self, level, dt, r, sigma):
        self.level = level #tree level starting from 0
        self.t = level / 2 #time = 0, 0.5, ...
        self.dt = dt #discount factor
        self.r = r #interest rate
        self.sigma = sigma #sigma
        self.prev = None
        self.next = None
        self.rPath = []
        self.forwardRate = 0.0
    
    def buildTree(self, depth):
        curr = self
        for i in range(depth):
            temp = curr
            curr.next = node(curr.level + 1, dtdf.loc[i + 1, 0], 0, sigmadf.loc[i, 0])
            curr = curr.next
            curr.prev = temp
        return curr

    def setR(self):
        # no need to solve for first node
        curr = self.next
        while curr:
            curr.r = fsolve(func, 0.06, args  = curr)[0]
            curr.rPath.append(curr.r)
            for i in range(curr.level):
                curr.rPath.append(curr.rPath[i] * np.exp(2 * curr.sigma * np.sqrt(0.5)))
            curr = curr.next

    def printAllR(self):
        curr = self
        while curr:
            print(f'level:{curr.level} rate:{curr.rPath}')
            curr = curr.next
    
    def setForwardRate(self):
        curr = self
        # need to stop at second last node or we go out of scope
        while curr.next:
            curr.forwardRate = (curr.dt / curr.next.dt - 1) * 2
            curr = curr.next

In [6]:
# initialize root
root = node(0, dtdf.loc[0, 0], (1 / dtdf.loc[0, 0] - 1) * 2, 0)

# build list to store all data first r is not calculated at this point
tail = root.buildTree(29)
root.setForwardRate()
root.rPath.append(root.r)

root.setR()

In [7]:
# use df to contain the outputs
allRatePath = pd.DataFrame(index = [i for i in range(30)])
averageRate = pd.DataFrame(index = [i / 2 for i in range(30)])

curr = root
normalAverage = []
pasAverage = []
forward = []
for i in range(30):
    if curr:
        temp = pd.Series(curr.rPath)
        allRatePath[f'{i/2}'] = temp
        normalAverage.append(np.mean(curr.rPath))
        pasAverage.append(pascalTriWeightedAverage(curr.rPath))
        forward.append(curr.forwardRate)
        curr = curr.next

averageRate['Average 6m rate'] = normalAverage
averageRate['Pascal Tri Weighted Avg 6m rate'] = pasAverage
averageRate['Forward 6m rate'] = forward
averageRate

,Average 6m rate,Pascal Tri Weighted Avg 6m rate,Forward 6m rate
0.0,0.056605,0.056605,0.059090
0.5,0.059098,0.059098,0.061650
1.0,0.061834,0.061687,0.063946
1.5,0.064624,0.064044,0.065585
2.0,0.067262,0.065791,0.066575
2.5,0.069781,0.066938,0.066992
3.0,0.071965,0.067538,0.067204
3.5,0.074398,0.067972,0.067484
4.0,0.076964,0.068486,0.067916
4.5,0.080000,0.069181,0.068563


In [53]:
# output to excel
with pd.ExcelWriter('output.xlsx') as writer:  
    allRatePath.to_excel(writer, sheet_name='BDT rate tree')
    averageRate.to_excel(writer, sheet_name='Average rate vs Forward rate')


In [ ]:
# test

# # print(tail.sigma.values, tail.dt.values)
# def myfunc(x):
#     y = root.dt * 0.5 * (1 / (1 + x / 2) + 1 / (1 + x * np.exp(2 * root.next.sigma * np.sqrt(0.5)) / 2)) - root.next.dt
#     return y
# sol = fsolve(myfunc, 0.06)
# print(sol)